# ECE 209 Mobile Security CA 4
Implement a simple remote attestation protocol, an attack scenario, and introduce noisy communication to study the impact of network and processing noise.


# Remote Attestation and Response Time

In [1]:
# file path
zero_8_path = '/content/zeros_8.txt'
attack_8_path = '/content/attack_8.txt'
mem_path = '/content/random_numbers_16.txt'


In [2]:
# verifier function
import random
from hashlib import sha256

# hash function
def hash_f (pre_hash, new_line):
  text = pre_hash + new_line
  return sha256(text.encode('utf-8')).hexdigest()

# generate a list of non-repeated nonce in a range of (0,256)
def verifier_send_nonce (multi_att):
  nonce_list = random.sample(range(257), multi_att)
  return nonce_list

# verifying process
def verifier_end (mem_path, nonce, hash_prover):
  hash_verif = hash_f('', nonce)
  # hash each line with hash from the previous line
  with open(mem_path, 'r') as file:
    for line in file:
      hash_verif = hash_f(hash_verif, line)
  # compare hash value with prover
  return 1 if hash_verif == hash_prover else 0


In [3]:
# prover function
from hashlib import sha256

# receive nonce and response
def prover_end(mem_path, mem_path_2, nonce):
  hash_prover = hash_f('', nonce)
  # hash each line with hash from the previous line
  with open(mem_path, 'r') as file:
    for line in file:
      hash_prover = hash_f(hash_prover, line)
  return hash_prover


In [4]:
# remote attestation
import time

# multi_att : numer of attestation
def remote_att(ver_path, pro_path_1, pro_path_2, multi_att, verifier, prover, print_status=1):
  # generate a list of non-repeat nonce for multiple attestation
  nonce_list = verifier_send_nonce(multi_att)
  time_list = []
  for index, nonce in enumerate(nonce_list):
    time_start = time.perf_counter() # time start
    hash_prover = prover(pro_path_1, pro_path_2, str(nonce))
    time_end = time.perf_counter() # time end
    time_list.append(time_end - time_start)

    att_result = verifier(ver_path, str(nonce), hash_prover)
    if(print_status):
      print(f'attestation run {index}, {("success" if att_result == 1 else "fail")}')

  return time_list

# attestation with 10 times testing
att_time = remote_att(mem_path, mem_path, _, 10, verifier_end, prover_end)

# avg attestation time
print('avg time attestation:', sum(att_time) / len(att_time))

attestation run 0, success
attestation run 1, success
attestation run 2, success
attestation run 3, success
attestation run 4, success
attestation run 5, success
attestation run 6, success
attestation run 7, success
attestation run 8, success
attestation run 9, success
avg time attestation: 0.0028974802999812256


# Malicous_prover Attack and Respond Time

In [5]:
# pre_compute function.
# Replace zero part of f2_file (second half) with non_zero part of f1_text (first half)

def pre_compute(file_path_1, file_path_2):
  # list of file 1
  with open(file_path_1, 'r') as f1:
    lines = f1.readlines()
    f1_text = [line for line in lines]

  # list of file 2
  with open(file_path_2, 'r') as f2:
    lines = f2.readlines()
    f2_text = [line for line in lines]

  # combine to new_file
  new_file = []
  half_size = int(len(f2_text)/2)

  for x in range(half_size):
    new_file.append(f2_text[x])
  for y in range(half_size):
    new_file.append(f1_text[y])
  print('combine file complete.')

  # verify the new file size
  print('file size:', len(new_file))

  # write out the file
  with open('pre_compute.txt', 'w') as file:
    for item in new_file:
      file.write(item)

  # return the pre_compute file as a list
  return new_file

# computation
pre_compute_file = pre_compute(zero_8_path, attack_8_path)
pre_compute_path = '/content/pre_compute.txt'


combine file complete.
file size: 1600


In [6]:
# malicious prover function
from hashlib import sha256

def malicous_prover(f1, f2, nonce):
  hash_prover = hash_f('', nonce)
  # attack_8 file
  with open(f1, 'r') as f1:
    lines = f1.readlines()
    f1_text = [line for line in lines]

  # pre_compute file
  with open(f2, 'r') as f2:
    lines = f2.readlines()
    f2_text = [line for line in lines]

  # hash the section where pre_compute is diff from attack_8
  # the un_overlapping section is zero_8.txt
  idx_count = 0
  tag = 0
  for attack, pre_compute in zip(f1_text, f2_text):
    if (attack != pre_compute):
      tag = 1
    if (tag == 1):
      hash_prover = hash_f(hash_prover, pre_compute)
      idx_count += 1

  for _ in range(idx_count, len(f2_text)):
    hash_prover = hash_f(hash_prover, '0\n')

  return hash_prover


# attack with attack_8.txt and pre_compute.txt
att_time = remote_att(zero_8_path, attack_8_path, pre_compute_path, 10, verifier_end, malicous_prover)

# avg attestation time
print('avg time malicious attack:', sum(att_time) / len(att_time))

attestation run 0, success
attestation run 1, success
attestation run 2, success
attestation run 3, success
attestation run 4, success
attestation run 5, success
attestation run 6, success
attestation run 7, success
attestation run 8, success
attestation run 9, success
avg time malicious attack: 0.0029932494000092904


# Verifier Identify Attack with Threshold Response Time

The way of finding threshold response time.

The response time begins when the prover receives a nonce and concludes with the prover hashing all lines of the specified mem_file. Given that the verifier has access to the identical mem_file, it's feasible to replicate the entire process on the verifier's side. This enables the collection of multiple response times, allowing for the determination of a threshold by calculating the mean + 4std (4σ), which encompasses 99.9937% of the outputs in a Gaussian distribution. In practice, this procedure is executed 200 times, with each response time being recorded in a time_list. The mean and standard deviation are then computed, and the threshold is established at mean + 4σ.

Consequently, the verifier adopts a dynamic threshold instead of a static one, enhancing the defense against similar types of attacks.

In [23]:
# verifier with threshold response time
import numpy as np
import time

def verifier_detect(mem_path, nonce, hash_prover, response_time, print_status=1, noise=0):
  # verify as normal
  ver_result = verifier_end(mem_path, nonce, hash_prover)

  # find threshold
  # hashing on given mem_file multiple times, 200 times in this case
  time_list = []
  for _ in range(200):
    time_start = time.perf_counter() # time start
    hash_verif = hash_f('', nonce)
    # hash each line with hash from the previous line
    with open(mem_path, 'r') as file:
      for line in file:
        hash_verif = hash_f(hash_verif, line)
    time_end = time.perf_counter() # time end
    time_list.append(time_end - time_start)

  # plot the list of response time in a gaussian distribution
  # take mean+4_sigma as the threshold
  mean = np.mean(time_list)
  std = np.std(time_list)
  thres_time = mean + std * 4
  if(print_status):
    print('threshold:', thres_time)

  # check response time
  if (response_time <= thres_time):
    return ver_result
  else:
    return 0


In [24]:
# verifier detection function

def detect_att(ver_path, pro_path_1, pro_path_2, multi_att, verifier, prover, print_status=1, noise=0.0):
  # generate a list of non-repeat nonce for multiple attestation
  nonce_list = verifier_send_nonce(multi_att)
  result_list = []
  for index, nonce in enumerate(nonce_list):
    time_start = time.perf_counter() # time start
    hash_prover = prover(pro_path_1, pro_path_2, str(nonce))
    time_end = time.perf_counter() # time end
    response_time = time_end - time_start

    # simulate noisy environment
    response_time += noise

    att_result = verifier(ver_path, str(nonce), hash_prover, response_time, print_status, noise)
    result_list.append(att_result)
    if(print_status):
      print('response time:', response_time)
      print(f'attestation run {index}, {("success" if att_result == 1 else "fail")}')
      print('-'*20)

  return result_list


In [25]:
# TP/FP rate calculation

def tp_fp_rate(num_list):
  p_count = 0
  for x in num_list:
    if x == 1:
      p_count+=1
  # rate
  p_rate = p_count / len(num_list)
  return p_rate

In [26]:
# 5 times with correct prover
total_count = 5
cor_list = detect_att(mem_path, mem_path, _, total_count, verifier_detect, prover_end)

# true positive rate
tp_rate = tp_fp_rate(cor_list)
print('true positive rate:', tp_rate)

threshold: 0.006638561305918249
response time: 0.0024397610000050918
attestation run 0, success
--------------------
threshold: 0.018212537701380267
response time: 0.0038428249999924446
attestation run 1, success
--------------------
threshold: 0.005620668722575069
response time: 0.0037822179999693617
attestation run 2, success
--------------------
threshold: 0.004416841362775131
response time: 0.006233756000028734
attestation run 3, fail
--------------------
threshold: 0.003119398360894042
response time: 0.0028326970000307483
attestation run 4, success
--------------------
true positive rate: 0.8


In [29]:
# 5 times with malicious prover
total_count = 5
mal_list = detect_att(zero_8_path, attack_8_path, pre_compute_path, total_count, verifier_detect, malicous_prover)

# false positive rate
fp_rate = tp_fp_rate(mal_list)
print('false positive rate:', fp_rate)


threshold: 0.0027185543049858354
response time: 0.005153049000000465
attestation run 0, fail
--------------------
threshold: 0.004174881639036963
response time: 0.004259133999994447
attestation run 1, fail
--------------------
threshold: 0.003986382278890572
response time: 0.0038458390001778753
attestation run 2, success
--------------------
threshold: 0.0034365683679862882
response time: 0.0031588930000907567
attestation run 3, success
--------------------
threshold: 0.003551585088459249
response time: 0.006652108000025692
attestation run 4, fail
--------------------
false positive rate: 0.4


# Mimic Noise in Timer

In [30]:
# random gaussian noise to response time
# noise in a order of 10e-3
import numpy as np

def noisy_att(ver_path, pro_path_1, pro_path_2, verifier, prover):
  # the average response time at 0.003s
  # expect max_noise = half of avg response time = 0.0015s
  max_noise = 0.0015
  noise_list = np.linspace(0, max_noise, 10)
  for noise in noise_list:
    print('noise_level:',noise)
    att_list = detect_att(ver_path, pro_path_1, pro_path_2, 50, verifier, prover, 0, noise)
    tp_rate = tp_fp_rate(att_list)

    # if tp rate drop to 0.5, done
    if (tp_rate <= 0.5):
      print('Thats it.')
      print('Given noise level:', noise)
      print('TP rate goes down to:', tp_rate)
      return

  # if no result from the given noise range
  print('noise too low to get tp rate down to 0.5')


In [18]:
# implement
noisy_att(mem_path, mem_path, _, verifier_detect, prover_end)

noise_level: 0.0
noise_level: 0.00016666666666666666
noise_level: 0.0003333333333333333
noise_level: 0.0005
noise_level: 0.0006666666666666666
noise_level: 0.0008333333333333333
noise_level: 0.001
Thats it.
Given noise level: 0.001
TP rate goes down to: 0.5


# Improve True Positive Rate in Presence of Noise

The verifier can preprocess an empty file to collect response times that are corrupted by noise and calculate the noise level that should be subtracted from the response times received from the prover. I collect multiple corrupted response times to calculate the mean + 4*std, treating it as random noise.

In [19]:
# create an empty file
with open('empty.txt', 'w') as file:
   pass

In [34]:
# modified verifier function

import numpy as np
import time

def verifier_detect_noiseless(mem_path, nonce, hash_prover, response_time, print_status=1, noise=0):
  # verify as normal
  ver_result = verifier_end(mem_path, nonce, hash_prover)

  # preprocess an empty file to estimate noise
  noise_list = []
  for _ in range(10):
    noise_time_start = time.perf_counter()
    with open('empty.txt', 'r') as f:
      for line in f:
        noise_hash = hash_f('','')
    noise_time_end = time.perf_counter()
    noise_response_time = (noise_time_end - noise_time_start) + noise
    noise_list.append(noise_response_time)

  # estimate upcoming noise with gaussian distribution
  noise_mean = np.mean(noise_list)
  noise_std = np.std(noise_list)
  noise_expect = noise_mean + noise_std * 4

  # find threshold
  # hashing on given mem_file multiple times, 200 times in this case
  time_list = []
  for _ in range(200):
    time_start = time.perf_counter() # time start
    hash_verif = hash_f('', nonce)
    # hash each line with hash from the previous line
    with open(mem_path, 'r') as file:
      for line in file:
        hash_verif = hash_f(hash_verif, line)
    time_end = time.perf_counter() # time end
    time_list.append(time_end - time_start)

  # plot the list of response time in a gaussian distribution
  # take mean+4_sigma as the threshold
  mean = np.mean(time_list)
  std = np.std(time_list)
  thres_time = mean + std * 4
  if(print_status):
    print('threshold:', thres_time)

  # check response time
  if ((response_time - noise_expect) <= thres_time):
    return ver_result
  else:
    return 0

In [35]:
# implement
noisy_att(mem_path, mem_path, _, verifier_detect_noiseless, prover_end)

noise_level: 0.0
noise_level: 0.00016666666666666666
noise_level: 0.0003333333333333333
noise_level: 0.0005
noise_level: 0.0006666666666666666
noise_level: 0.0008333333333333333
noise_level: 0.001
noise_level: 0.0011666666666666665
noise_level: 0.0013333333333333333
noise_level: 0.0015
noise too low to get tp rate down to 0.5


The performance does improve. A noise level of 0.0015 seconds is required to lower the TP (True Positive) rate to 0.5, compared to a noise level of 0.001 seconds before.